rendering placeholder

In [ ]:
# ============================================================
# UNet Road Segmentation for Your TIFF Dataset
# Structure copied as closely as possible from Massachusetts NB
# ============================================================

import os, cv2
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album

import segmentation_models_pytorch as smp
from pathlib import Path
import rasterio

# ============================================================
# Dataset Paths 📁
# ============================================================

DATA_DIR = Path("data/tiffs")

# We artificially create train/val splits
all_tifs = sorted(list(DATA_DIR.glob("*.tif")) + list(DATA_DIR.glob("*.tiff")))
random.seed(42)
random.shuffle(all_tifs)

train_split = int(0.8 * len(all_tifs))
val_split   = int(0.9 * len(all_tifs))

train_files = all_tifs[:train_split]
val_files   = all_tifs[train_split:val_split]
test_files  = all_tifs[val_split:]

print("Train:", len(train_files))
print("Val:", len(val_files))
print("Test:", len(test_files))

# ============================================================
# Class definitions (same style as the Massachusetts NB)
# ============================================================

class_names = ["background", "road"]
class_rgb_values = [[0,0,0], [255,255,255]]  # for visualization
select_class_rgb_values = np.array(class_rgb_values)

print("All dataset classes:", class_names)
print("Class RGB map:", class_rgb_values)

# ============================================================
# Helper Functions
# ============================================================

def visualize(**images):
    n = len(images)
    plt.figure(figsize=(20,8))
    for i, (name, img) in enumerate(images.items()):
        plt.subplot(1, n, i+1)
        plt.title(name.replace("_"," ").title(), fontsize=18)
        plt.imshow(img)
        plt.axis("off")
    plt.show()

def one_hot_encode(mask):
    # mask is binary (0/255). convert to 0/1 then stack channels
    road = (mask > 0).astype(np.uint8)
    background = 1 - road
    return np.stack([background, road], axis=-1).astype("float32")

def reverse_one_hot(y):
    return np.argmax(y, axis=-1)

def colour_code_segmentation(mask):
    return np.array(class_rgb_values)[mask.astype(int)]

# ============================================================
# Dataset Class (adapted for TIFF RGB+DEM+MASK)
# ============================================================

class RoadsDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, augmentation=None, preprocessing=None):
        self.files = file_list
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):
        tif = self.files[i]
        with rasterio.open(tif) as src:
            arr = src.read()        # [bands, H, W]
        rgb = np.moveaxis(arr[:3], 0, -1).astype(np.uint8)
        mask = arr[-1].astype(np.uint8)

        # one-hot encode
        mask = one_hot_encode(mask)

        # Albumentations needs HWC
        if self.augmentation:
            sample = self.augmentation(image=rgb, mask=mask)
            rgb, mask = sample["image"], sample["mask"]

        if self.preprocessing:
            sample = self.preprocessing(image=rgb, mask=mask)
            rgb, mask = sample["image"], sample["mask"]

        return rgb, mask

    def __len__(self):
        return len(self.files)

# ============================================================
# Augmentations
# ============================================================

def get_training_augmentation():
    return album.Compose([
        album.RandomCrop(height=256, width=256, always_apply=True),
        album.OneOf([
            album.HorizontalFlip(p=1),
            album.VerticalFlip(p=1),
            album.RandomRotate90(p=1),
        ], p=0.75),
    ])

def get_validation_augmentation():
    return album.Compose([])

def to_tensor(x, **kwargs):
    return x.transpose(2,0,1).astype("float32")

def get_preprocessing(pre_fn):
    return album.Compose([
        album.Lambda(image=pre_fn),
        album.Lambda(image=to_tensor, mask=to_tensor),
    ])

# ============================================================
# Visualize one example
# ============================================================

dataset_vis = RoadsDataset(train_files, augmentation=None)
rgb, mask = dataset_vis[0]

visualize(
    original_image = rgb,
    ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask)),
    one_hot_encoded_mask = reverse_one_hot(mask)
)

# ============================================================
# Create Datasets & Loaders
# ============================================================

ENCODER = "resnet50"
ENCODER_WEIGHTS = "imagenet"
CLASSES = ["background", "road"]
ACTIVATION = "sigmoid"

model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=2,
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

train_dataset = RoadsDataset(
    train_files,
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
)

val_dataset = RoadsDataset(
    val_files,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4)

# ============================================================
# Training Setup
# ============================================================

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(DEVICE)

loss = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU(threshold=0.5)]

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

train_epoch = smp.utils.train.TrainEpoch(
    model, loss=loss, metrics=metrics,
    optimizer=optimizer, device=DEVICE, verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, loss=loss, metrics=metrics,
    device=DEVICE, verbose=True
)

# ============================================================
# Train UNet
# ============================================================

EPOCHS = 5
best_iou = 0
train_logs_list, val_logs_list = [], []

for epoch in range(EPOCHS):
    print("\nEpoch", epoch)
    train_logs = train_epoch.run(train_loader)
    val_logs   = valid_epoch.run(val_loader)

    train_logs_list.append(train_logs)
    val_logs_list.append(val_logs)

    if val_logs["iou_score"] > best_iou:
        best_iou = val_logs["iou_score"]
        torch.save(model.state_dict(), "best_unet.pth")
        print("Model saved!")

# ============================================================
# Prediction on Test Set
# ============================================================

model.load_state_dict(torch.load("best_unet.pth"))
model.eval()

test_dataset = RoadsDataset(
    test_files,
    augmentation=None,
    preprocessing=get_preprocessing(preprocessing_fn)
)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

test_vis = RoadsDataset(test_files)  # no preprocessing

# Show one example
idx = 0
x, y = test_vis[idx]
x_tensor = torch.tensor(x.transpose(2,0,1)).unsqueeze(0).float().to(DEVICE)
pred = model(x_tensor).detach().cpu().numpy().squeeze().transpose(1,2,0)
pred_mask = reverse_one_hot(pred)
gt_mask = reverse_one_hot(y)

visualize(
    original_image=x,
    ground_truth_mask=colour_code_segmentation(gt_mask),
    predicted_mask=colour_code_segmentation(pred_mask)
)

# ============================================================
# Evaluate on Test Set
# ============================================================

test_epoch = smp.utils.train.ValidEpoch(
    model, loss=loss, metrics=metrics, device=DEVICE, verbose=True
)

test_logs = test_epoch.run(test_loader)
print("Test IoU:", test_logs["iou_score"])
print("Test Dice:", test_logs["dice_loss"])

# ============================================================
# Plot Loss / IoU Curves
# ============================================================

df_train = pd.DataFrame(train_logs_list)
df_val   = pd.DataFrame(val_logs_list)

plt.figure(figsize=(20,8))
plt.plot(df_train.index, df_train.iou_score, label="Train", lw=3)
plt.plot(df_val.index, df_val.iou_score, label="Val", lw=3)
plt.title("IoU per Epoch", fontsize=20)
plt.grid(); plt.legend(); plt.show()

plt.figure(figsize=(20,8))
plt.plot(df_train.index, df_train.dice_loss, label="Train", lw=3)
plt.plot(df_val.index, df_val.dice_loss, label="Val", lw=3)
plt.title("Dice Loss per Epoch", fontsize=20)
plt.grid(); plt.legend(); plt.show()